# Hertzsprung-Russell Diagram

> The Hertzsprung–Russell diagram, abbreviated H–R diagram or HRD, is a scatter graph of stars showing the relationship between the stars' absolute magnitudes or luminosities versus their spectral classifications or effective temperatures. More simply, it plots each star on a graph measuring the star's brightness against its temperature (color). It does not map any locations of stars.
>
> --<cite>[Wikipedia](https://en.wikipedia.org/wiki/Hertzsprung%E2%80%93Russell_diagram)</cite>

Based on [zonination's original work](https://www.reddit.com/r/dataisbeautiful/comments/433960).

Data from the [HYG star database archive](https://github.com/astronexus/HYG-Database).

## Step 1: Download the data and clean it

In [2]:
!curl -O https://raw.githubusercontent.com/astronexus/HYG-Database/master/hygdata_v3.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.9M  100 31.9M    0     0  1927k      0  0:00:16  0:00:16 --:--:-- 2123k


In [6]:
!ls -l | grep hyg

-rw-r--r--. 1 rober rober 33449663 May  5 10:42 hygdata_v3.csv


In [7]:
import pandas as pd

# The H-R diagram only shows the absolute magnitude and the color index.
# Every other column is discarded, and the ones with null values are dropped.
df = pd.read_csv('hygdata_v3.csv')[['absmag', 'ci']]
df.dropna(inplace=True) # drops 1882 rows

print '%i total rows' % len(df)
df.head(3)

117732 total rows


,absmag,ci
0,4.850,0.656
1,2.390,0.482
2,5.866,0.999
